In [1]:
import numpy as np
# Load the "autoreload" extension so that code can change
%load_ext autoreload

#always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.data import make_dataset

In [2]:
path = "/Users/jbaripatti/machine_learning_proj/data/interim/"
name = 'data_interim.txt'
filepath = path + name
data = np.loadtxt(filepath, delimiter=",")
X_train, y_train, X_test, y_test = make_dataset.split_data(data)

In [3]:
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.models import Sequential

Using TensorFlow backend.


In [26]:
np.random.seed(123)

In [18]:
X_train, X_test = make_dataset.scale_data(X_train, X_test)

### We have 50 input features and one target variable. 
### 2 Hidden layers. Each hidden layer will have 25 nodes.

ReLu will be the activation function for hidden layers. As this is a binary classification problem we will use sigmoid as the activation function.

In [19]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(25, activation='relu', kernel_initializer='random_normal', input_dim=50))
#Second  Hidden Layer
classifier.add(Dense(25, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [20]:
#Compiling the neural network
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [21]:
#Fitting the data to the training dataset
classifier.fit(X_train,y_train, batch_size=10, epochs=100)

Epoch 1/100
104051/104051 [==============================] - 11s 103us/step - loss: 0.2337 - acc: 0.9084
Epoch 2/100
104051/104051 [==============================] - 10s 96us/step - loss: 0.1823 - acc: 0.9254
Epoch 3/100
104051/104051 [==============================] - 10s 97us/step - loss: 0.1731 - acc: 0.9298
Epoch 4/100
104051/104051 [==============================] - 11s 103us/step - loss: 0.1687 - acc: 0.9311
Epoch 5/100
104051/104051 [==============================] - 10s 98us/step - loss: 0.1655 - acc: 0.9327 0s - loss: 0.1654 - ac
Epoch 6/100
104051/104051 [==============================] - 10s 99us/step - loss: 0.1626 - acc: 0.9341 0s - loss: 0.1624 -
Epoch 7/100
104051/104051 [==============================] - 11s 101us/step - loss: 0.1774 - acc: 0.9352
Epoch 8/100
104051/104051 [==============================] - 10s 97us/step - loss: 0.2133 - acc: 0.9331
Epoch 9/100
104051/104051 [==============================] - 11s 101us/step - loss: 0.2118 - acc: 0.9338
Epoch 10/100
1040

104051/104051 [==============================] - 9s 90us/step - loss: 0.1344 - acc: 0.9471
Epoch 78/100
104051/104051 [==============================] - 9s 89us/step - loss: 0.1343 - acc: 0.9474
Epoch 79/100
104051/104051 [==============================] - 9s 89us/step - loss: 0.1340 - acc: 0.9472
Epoch 80/100
104051/104051 [==============================] - 9s 90us/step - loss: 0.1343 - acc: 0.9470
Epoch 81/100
104051/104051 [==============================] - 9s 89us/step - loss: 0.1342 - acc: 0.9470
Epoch 82/100
104051/104051 [==============================] - 10s 94us/step - loss: 0.1338 - acc: 0.9471
Epoch 83/100
104051/104051 [==============================] - 9s 90us/step - loss: 0.1339 - acc: 0.9477
Epoch 84/100
104051/104051 [==============================] - 9s 89us/step - loss: 0.1334 - acc: 0.9475: 
Epoch 85/100
104051/104051 [==============================] - 9s 89us/step - loss: 0.1330 - acc: 0.9480
Epoch 86/100
104051/104051 [==============================] - 9s 89us/step

In [22]:
eval_model=classifier.evaluate(X_train, y_train)
eval_model

104051/104051 [==============================] - 1s 11us/step


[0.12995500298622456, 0.948717455860126]

In [23]:
y_pred=classifier.predict(X_test)
y_pred =(y_pred>0.5)

In [24]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[18658     0]
 [ 7355     0]]


In [25]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.717256756237266

## Grid Search Hyperparameters

In [30]:
from src.models import train_model
from keras.wrappers.scikit_learn import KerasClassifier


### Tune Batch Size and Number of Epochs

In [31]:
#initialize model
train_model.create_model_batch_epoch()

In [33]:
# create model
model = KerasClassifier(build_fn=train_model.create_model, verbose=0)

# define the grid search parameters
batch_size = [10, 100, 1000]
epochs = [10, 50, 100]

param_grid = dict(batch_size=batch_size, epochs=epochs)

In [34]:
train_model.get_best_result(param_grid, model, X_train, y_train)

/anaconda3/envs/finalprojectML/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/anaconda3/envs/finalprojectML/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best: 0.936973 using {'batch_size': 10, 'epochs': 100}
0.929832 (0.001010) with: {'batch_size': 10, 'epochs': 10}
0.935282 (0.001335) with: {'batch_size': 10, 'epochs': 50}
0.936973 (0.001509) with: {'batch_size': 10, 'epochs': 100}
0.924720 (0.001334) with: {'batch_size': 100, 'epochs': 10}
0.932427 (0.000978) with: {'batch_size': 100, 'epochs': 50}
0.936839 (0.001878) with: {'batch_size': 100, 'epochs': 100}
0.896061 (0.002757) with: {'batch_size': 1000, 'epochs': 10}
0.927920 (0.000533) with: {'batch_size': 1000, 'epochs': 50}
0.931553 (0.001205) with: {'batch_size': 1000, 'epochs': 100}


### Tune Learning Rate

In [35]:
#initizalize model
train_model.create_model_lrate()

In [37]:
model = KerasClassifier(build_fn=train_model.create_model_lrate, epochs=50, batch_size=10, verbose=0)

# define the grid search parameters
learn_rate = [0.001, 0.2, 0.9]
param_grid = dict(learn_rate=learn_rate)

In [38]:
train_model.get_best_result(param_grid, model, X_train, y_train)

KeyboardInterrupt: 